<a href="https://colab.research.google.com/github/NaviaEddy/SegundoParcial_SIS421/blob/main/TransformerNLP_NaviaCondoriEddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 30.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim #que proporciona varios algoritmos de optimización (como SGD, Adam, etc.)
                            # utilizados para actualizar los pesos de la red neuronal durante el entrenamiento

import spacy #es una biblioteca de procesamiento de lenguaje natural (NLP)
             # para cargar modelos de lenguaje, tokenizar texto, etc.

import warnings #que permite emitir advertencias en Python.

from torch.utils.tensorboard import SummaryWriter # se utiliza para escribir datos de entrenamiento (como la pérdida y la precisión) en un archivo

from sklearn.model_selection import train_test_split #Esta función se utiliza para dividir un conjunto de datos

from torch.nn.utils.rnn import pad_sequence # se utiliza para rellenar secuencias de diferentes longitudes a la misma longitud

from torch.utils.data import DataLoader, Dataset

In [43]:
# Supresión de advertencias específicas
warnings.filterwarnings("ignore", message="enable_nested_tensor is True, but self.use_nested_tensor is False because")
warnings.filterwarnings("ignore", message="The verbose parameter is deprecated.")

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
import os
import pandas as pd

# Descargamos el dataset
!wget https://www.manythings.org/anki/spa-eng.zip -O spa-eng.zip
!unzip spa-eng.zip -d spa-eng

# Cargamos el dataset
data_path = 'spa-eng/spa.txt'

# Lee el conjunto de datos y conserva sólo las dos primeras columnas
df = pd.read_csv(data_path, delimiter='\t', header=None, names=['src', 'trg', '_extra1', '_extra2', '_extra3'])

# Eliminar las columnas adicionales
df = df[['src', 'trg']]

# Mostramos las filas
print(df.head())

# Guardamos el dataset tratado
df.to_csv('spa-eng/spa_processed.csv', index=False)

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [55]:
# Carga los datos desde un archivo CSV
df = pd.read_csv('spa-eng/spa_processed.csv')

# Extrae todas las frases en español de la columna 'src' del DataFrame y las convierte en una lista de cadenas de texto.
src_sentences = df['src'].tolist()

# Extrae todas las frases en inglés de la columna 'trg' del DataFrame y las convierte en una lista de cadenas de texto.
trg_sentences = df['trg'].tolist()

In [54]:
import spacy
from torchtext.data.metrics import bleu_score

def translate_sentence(model, sentence, src_vocab, trg_vocab, reverse_trg_vocab, device, max_length=50):
    # Carga el modelo de spaCy para el idioma español
    spacy_es = spacy.load("es_core_news_sm")

    # Tokeniza la oración si es una cadena de texto
    if isinstance(sentence, str):
        # Convierte la oración en una lista de tokens en minúsculas
        tokens = [token.text.lower() for token in spacy_es(sentence)]
    else:
        # Si la oración ya es una lista de tokens, solo los convierte a minúsculas
        tokens = [token.lower() for token in sentence]

    # Añade los tokens de inicio de oración (SOS) y fin de oración (EOS)
    tokens.insert(0, "<sos>")
    tokens.append("<eos>")

    # Convierte los tokens en índices según el vocabulario de origen, si no se
    # encuentra la palabra devuelve <unk>
    text_to_indices = [src_vocab.get(token, src_vocab["<unk>"]) for token in tokens]

    # Convierte la lista de índices en un tensor y lo añade una dimensión extra para batch
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Inicializa la lista de salidas con el token de inicio de oración del vocabulario de destino
    outputs = [trg_vocab["<sos>"]]

    # Traduce la oración token por token hasta el máximo de longitud permitido
    for _ in range(max_length):
        # Convierte la lista de salidas en un tensor y lo añade una dimensión extra para batch
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        # Desactiva el cálculo de gradientes para ahorrar memoria y acelerar la inferencia
        with torch.no_grad():
            # Pasa el tensor de la oración y el tensor de las salidas al modelo para obtener el siguiente token
            output = model(sentence_tensor, trg_tensor)

        # Selecciona el índice del token con la mayor probabilidad en la última predicción
        best_guess = output.argmax(2)[-1, :].item()

        # Añade el índice del mejor token a la lista de salidas
        outputs.append(best_guess)

        # Si el token de fin de oración (EOS) es predicho, se detiene la traducción
        if best_guess == trg_vocab["<eos>"]:
            break

    # Convierte los índices a sentencias
    translated_sentence = [reverse_trg_vocab[token] for token in outputs[1:]]

    # Retorna la oración traducida como una lista de tokens
    return translated_sentence

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])


In [48]:
# Descarga y carga de modelos de spacy
spacy.require_gpu()  # Indica a spacy que use la GPU si está disponible para acelerar el procesamiento
spacy_es = spacy.load("es_core_news_sm")  # Carga el modelo de spacy para procesar texto en español
spacy_en = spacy.load("en_core_web_sm")  # Carga el modelo de spacy para procesar texto en inglés

# Definición de funciones de tokenización
def tokenize_es(text):
    # Tokeniza el texto en español utilizando el modelo de spacy para español
    return [tok.text for tok in spacy_es.tokenizer(text)]

def tokenize_en(text):
    # Tokeniza el texto en inglés utilizando el modelo de spacy para inglés
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [49]:
# Definición de un dataset personalizado para traducción
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, trg_sentences, src_tokenizer, trg_tokenizer, src_vocab, trg_vocab):
        # Inicializa el dataset con las oraciones de origen y destino, así como los tokenizadores y vocabularios correspondientes
        self.src_sentences = src_sentences  # Lista de oraciones en el idioma de origen
        self.trg_sentences = trg_sentences  # Lista de oraciones en el idioma de destino
        self.src_tokenizer = src_tokenizer  # Función para tokenizar oraciones en el idioma de origen
        self.trg_tokenizer = trg_tokenizer  # Función para tokenizar oraciones en el idioma de destino
        self.src_vocab = src_vocab  # Vocabulario para el idioma de origen (diccionario de token a índice)
        self.trg_vocab = trg_vocab  # Vocabulario para el idioma de destino (diccionario de token a índice)

    def __len__(self):
        # Devuelve el número de ejemplos en el dataset
        return len(self.src_sentences)

    def __getitem__(self, idx):
        # Devuelve el par de tokens (oración de origen y oración de destino) en forma de tensores para un índice dado
        # Tokeniza la oración de origen y la convierte en índices según el vocabulario
        src_tokens = [self.src_vocab.get(token, self.src_vocab["<unk>"]) for token in self.src_tokenizer(self.src_sentences[idx])]
        # Tokeniza la oración de destino y la convierte en índices según el vocabulario
        trg_tokens = [self.trg_vocab.get(token, self.trg_vocab["<unk>"]) for token in self.trg_tokenizer(self.trg_sentences[idx])]
        # Convierte las listas de índices en tensores y los devuelve como una tupla
        return torch.tensor(src_tokens), torch.tensor(trg_tokens)

In [50]:
def build_vocab(tokenizer, sentences, min_freq=2):
    # Inicializa un diccionario vacío para almacenar las frecuencias de los tokens
    vocab = {}
    # Itera sobre cada oración en el conjunto de datos
    for sentence in sentences:
        # Tokeniza la oración usando el tokenizador proporcionado
        tokens = tokenizer(sentence)
        # Itera sobre cada token en la oración
        for token in tokens:
            # Incrementa la frecuencia del token en el vocabulario
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 1
    # Filtra el vocabulario para mantener solo los tokens que ocurren al menos min_freq veces
    vocab = {k: v for k, v in vocab.items() if v >= min_freq}
    # Asigna un índice único a cada token en el vocabulario, comenzando desde 4
    vocab = {token: idx for idx, token in enumerate(vocab.keys(), start=4)}
    # Agrega tokens especiales al vocabulario con índices predefinidos
    vocab["<unk>"] = 0  # Token para palabras desconocidas
    vocab["<pad>"] = 1  # Token de relleno para hacer que las secuencias tengan la misma longitud
    vocab["<sos>"] = 2  # Token de inicio de secuencia
    vocab["<eos>"] = 3  # Token de final de secuencia
    return vocab

In [57]:
# Separamos los
train_src, test_src, train_trg, test_trg = train_test_split(src_sentences, trg_sentences, test_size=0.1)
train_src, val_src, train_trg, val_trg = train_test_split(train_src, train_trg, test_size=0.1)

# Construimos los vocabularios
src_vocab = build_vocab(tokenize_es, train_src)
trg_vocab = build_vocab(tokenize_en, train_trg)

# Cambia de indices a sentencias
reverse_trg_vocab = {v: k for k, v in trg_vocab.items()}

# Creamos el dataset
train_dataset = TranslationDataset(train_src, train_trg, tokenize_es, tokenize_en, src_vocab, trg_vocab)
val_dataset = TranslationDataset(val_src, val_trg, tokenize_es, tokenize_en, src_vocab, trg_vocab)
test_dataset = TranslationDataset(test_src, test_trg, tokenize_es, tokenize_en, src_vocab, trg_vocab)

# Se encarga de manejar que todos los lotes tengan el mismo len agregando un pad
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=src_vocab["<pad>"])
    trg_batch = pad_sequence(trg_batch, padding_value=trg_vocab["<pad>"])
    return src_batch, trg_batch

# Creamos los dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [58]:
class Transformer(nn.Module):
    def __init__(self, embedding_size, src_vocab_size, trg_vocab_size, src_pad_idx, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion, dropout, max_len, device):
        super(Transformer, self).__init__()
        #Esta capa se utiliza para convertir los índices de las palabras en vectores de embedding(eng).
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        #Esta capa se utiliza para representar la posición de cada palabra en la secuencia.
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        #Esta capa se utiliza para convertir los índices de las palabras en vectores de embedding(esp).
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        #Esta capa se utiliza para representar la posición de cada palabra en la secuencia.
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.device = device
        # Capa principal del Transformer
        self.transformer = nn.Transformer(embedding_size, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion, dropout)
        #Crea una capa lineal que se utilizará como capa de salida del modelo.
        #Esta capa transforma la salida del Transformer en una representación adecuada para predecir las palabras en la secuencia objetivo.
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx #indice de padding

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx #Verifica que tokens llevan pad(true) y no pad(false)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        # (seq len, emdeding)
        src_seq_length, N = src.shape
        # (seq len, emdeding)
        trg_seq_length, N = trg.shape
        # Genera un tensor de posiciones para la fuente con una secuencia de 0 hasta seq len y añadiendole una dimension extra
        # para que este en el formato (seq len, emdeding)
        src_positions = torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N).to(self.device)
        trg_positions = torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device)
        #Suma el embeding y la posicion de representacion de embeding para cada fuente
        embed_src = self.dropout((self.src_word_embedding(src) + self.src_position_embedding(src_positions)))
        embed_trg = self.dropout((self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions)))
        #Se ocupa de ocultar el pad
        src_padding_mask = self.make_src_mask(src)
        #Evita que el modelo atienda los pad de los trg finales
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)
        #Se pasa las secuencias fuente y objetivo, junto con las máscaras para controlar la informacion
        out = self.transformer(embed_src, embed_trg, src_key_padding_mask=src_padding_mask, tgt_mask=trg_mask)
        #Obtenemos la distribucion de probabilidades para la prediccion
        out = self.fc_out(out)
        return out

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = True
save_model = True
num_epochs = 40
learning_rate = 3e-4
src_vocab_size = len(src_vocab)
trg_vocab_size = len(trg_vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = trg_vocab["<pad>"]

In [64]:
model = Transformer(embedding_size, src_vocab_size, trg_vocab_size, src_pad_idx, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion, dropout, max_len, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True)
pad_idx = trg_vocab["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [65]:
checkpoint_path = "my_checkpoint.pth.tar"
if load_model and os.path.exists(checkpoint_path):
    load_checkpoint(torch.load(checkpoint_path), model, optimizer)

sentence = "i want to go your favorite place please"

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint, filename=checkpoint_path)

    model.eval()
    translated_sentence = translate_sentence(model, sentence, src_vocab, trg_vocab, reverse_trg_vocab, device, max_length=30)
    print(f"Ejemplos de traducciones: \n {' '.join(translated_sentence)}")
    model.train()

    train_losses = []
    train_correct = 0
    train_total = 0

    for batch_idx, batch in enumerate(train_loader):
        inp_data, target = batch
        inp_data = inp_data.to(device)
        target = target.to(device)

        output = model(inp_data, target[:-1, :])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)
        train_losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

        # Calculate training accuracy
        _, predicted = output.max(1)
        train_total += target.size(0)
        train_correct += (predicted == target).sum().item()

    mean_train_loss = sum(train_losses) / len(train_losses)
    train_acc = 100 * train_correct / train_total

    model.eval()
    val_losses = []
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            inp_data, target = batch
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target[:-1, :])
            output = output.reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)

            loss = criterion(output, target)
            val_losses.append(loss.item())

            # Calculate validation accuracy
            _, predicted = output.max(1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_acc = 100 * val_correct / val_total

    scheduler.step(mean_val_loss)

    print(f"Train Loss: {mean_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {mean_val_loss:.4f}, Val Acc: {val_acc:.2f}%")

[Epoch 0 / 40]
=> Saving checkpoint
Ejemplos de traducciones: 
 imaginar chicas imaginar vecino convertirme criticado planeo imaginar criticado convertirme tienden interesan planeo planeo planeo planeo planeo planeo planeo planeo planeo planeo reaccionado literatura Les interesan imaginar habló Ponle reaccionado
Train Loss: 3.6514, Train Acc: 18.79%
Val Loss: 2.3822, Val Acc: 24.57%
[Epoch 1 / 40]
=> Saving checkpoint
Ejemplos de traducciones: 
 que quieres ir tu lugar , por favor . No <unk> . No ! <unk> . No . No . No . No ! <unk> . No . No .
Train Loss: 2.2836, Train Acc: 24.87%
Val Loss: 1.9515, Val Acc: 26.94%
[Epoch 2 / 40]
=> Saving checkpoint
Ejemplos de traducciones: 
 de <unk> que vaya tu lugar por favor . No <unk> más de <unk> . No <unk> . No . No por favor . No . No . No <unk>
Train Loss: 1.8748, Train Acc: 26.89%
Val Loss: 1.7790, Val Acc: 28.26%
[Epoch 3 / 40]
=> Saving checkpoint
Ejemplos de traducciones: 
 <unk> quiere ir tu sitio sitio por favor . A veces . A que <unk> 

In [68]:
# Cargar el checkpoint
checkpoint_path = "my_checkpoint.pth.tar"
if os.path.exists(checkpoint_path):
    load_checkpoint(torch.load(checkpoint_path), model, optimizer)

=> Loading checkpoint


In [69]:
# Frase para traducir
sentence = "i like you"

# Realizar la traducción
model.eval()
translated_sentence = translate_sentence(model, sentence, src_vocab, trg_vocab, reverse_trg_vocab, device, max_length=10)
translated_text = ' '.join(translated_sentence)
print(f"Traducción: {translated_text}")

Traducción: <unk> como te <unk> de que te <unk> a las
